In [ ]:
from pathlib import Path
import pandas as pd

In [ ]:
# Define I/O paths
path_input: Path = Path(
    "../../../data/original/other/HDL Proteome Watch 2023 Final.xlsx"
).resolve()
path_output: Path = Path(
    "../../../data/processed/other/hdl_proteome_davidson.csv"
).resolve()

In [ ]:
# Define columns to keep and rename
usecols_: list[str] = [
    "Recommended protein name",
    "Abbrev.",
    "Acc. #",
    "Hits",
    "Likely*",
]
colnames_: list[str] = [
    "protein_full_name",
    "abbreviation",
    "uniprot_id",
    "hits",
    "likely",
]

In [ ]:
# Input data
df: pd.DataFrame = pd.read_excel(
    path_input, sheet_name="4) Database - Frequency", header=9, usecols=usecols_
).convert_dtypes()

In [ ]:
# Rename columns and drop missing values
df: pd.DataFrame = (
    df.rename(columns=dict(zip(usecols_, colnames_)))
    .dropna()
    .drop_duplicates()
    .convert_dtypes()
    .reset_index(drop=True)
)

In [6]:
# Remove the '_HUMAN' suffix of the abbreviation
df["abbreviation"] = df["abbreviation"].str.split(pat="_").str[0]

In [7]:
# Entry at index 19 needs to be split into two rows
df.loc[19]

protein_full_name    serum amyloid A1 &2
abbreviation                   SAA1/SAA2
uniprot_id                 P0DJI8/P0DJI9
hits                                  43
likely                                 1
Name: 19, dtype: object

In [8]:
# Manually split the entry at index 19 into two rows
df.loc[len(df)] = ["serum amyloid A1", "SAA1", "P0DJI8", 43, 1]
df.loc[len(df)] = ["serum amyloid A2", "SAA2", "P0DJI9", 43, 1]

In [ ]:
# Drop the entry at index 19
df = (
    df.drop(index=19)
    .convert_dtypes()
    .sort_values(by=["hits", "abbreviation"], ascending=[False, True])
    .reset_index(drop=True)
)

In [10]:
# Save as CSV
df.to_csv(path_output, index=False)